#### ```0. Basic_check(df)```
- pd.sample 확인, info, 전체 데이터 수 및 null 값 현황을 확인하는 함수  
---
### 1. 날짜 관련 함수 처리
#### ```1-1. date_preprocess(df, date_colname, time_colname, new_colname)```
- tran = tran_date + tran_time
- play = play_date + play_st_time
- 시간 데이터 형식을 4자리 수로 변환 ex) 615->0615
- 날짜와 시간을 합쳐서 datetime 형식 데이터로 변경함
- 각 날짜별 요일을 한국어로 추출하여 tran_day, play_day col로 추가
- 함수 생성 후, 테스트를 위해 ```1000개``` 데이터만 사용함(df1000)
---
#### ```1-2. float_to_datetime(df, col_list)```
- float 타입의 데이터들을 datetime(yyyy-mm-dd 형식)으로 변경
- pre_open_date, open_date를 col_list로 전달
---
#### ```1-3. tran_rank_min(df)```
- 'performance_code'로 그룹화하여 'tran' 열을 가장 빠른 순서대로 정렬 후, rank를 매김
- 최종적으로 사용한 method = 'min' 이지만, method = 'first'와 비교하여 확인한 과정도 남겨두었음. 
- 특정 공연에서 빨리 예매된 좌석이 어디인지를 파악하고 그 좌석이 좋은 좌석이라는 feature로 사용하기 위한 작업
---
### ```2. 데이터 나누기```
### ```2-1. separate_seat(df)```
- 좌석 정보 분리하여 4개의 col에 저장 : seat_floor(층),seat_block(블록),seat_col(열),seat_num(좌석번호)
### ```2-2. discounts(df)```
- 할인명, 할인률 구분하기   
---

- Nan값 처리 [age, gender, 멤버십들, discount_type, open_date, pre_open_date, running_time, intermission]  
- dtype 변경

In [1]:
# !pip install -r requirements.txt
# !pip install pandas matplotlib seaborn

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 전체 데이터 출력 및 확인 옵션 설정
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_info_columns

100

# 0. basic_check
샘플 데이터 print, info, 전체 데이터 수 및 null 값 현황을 확인하는 함수

In [3]:
def basic_check(df):
    # 샘플 데이터 10개 확인
    print('샘플 데이터 10개 확인\n', df.sample(10))
    print('==================================================')
    # info 확인
    print('info 확인\n', df.info())
    print('==================================================')
    # 전체 데이터 수 및 null 값 확인
    print('전체 데이터 수 : ', len(df)) 
    print('null 값 확인\n', df.isnull().sum())

In [4]:
# 데이터셋 로드
df=pd.read_csv('./dataset/2023bigcontest_advanced_sac.csv')

# basic_check 수행
basic_check(df)

샘플 데이터 10개 확인
           age gender membership_type_1 membership_type_2 membership_type_3  \
653308   60.0      M                골드                무료               NaN   
1313627  60.0      M                골드                무료               NaN   
128352   40.0      F                블루                무료                그린   
1682779  60.0      F                블루                무료               NaN   
44206     NaN    NaN               NaN               NaN               NaN   
895073    NaN    NaN               NaN               NaN               NaN   
1457508  20.0      M                무료              싹틔우미               NaN   
578674    NaN    NaN               NaN               NaN               NaN   
1435868  30.0      F                무료              싹틔우미                그린   
572472    NaN    NaN               NaN               NaN               NaN   

        membership_type_4 membership_type_5  membership_type_6  tran_date  \
653308                NaN               NaN      

# 1-1. date_preprocess
* input
  - df : 데이터프레임
  - date_colname : 날짜 정보가 담긴 컬럼명 (yyyymmdd)
  - time_colname : 시간 정보가 담긴 컬럼명 (hhmm), 시간, 분을 4자리 수로 변환하는 작업 필요 ex) 615->0615
  - new_colname : 새롭게 생성할 컬럼명, date_colname + time_colname
* output
  - df : 데이터프레임
  - new_colname : date_colname + time_colname를 합친 후, datetime 형태로 가공(yyyy-mm-dd hh:mm:ss)
  - new_colname_day : 새롭게 생성된 컬럼에서 요일 정보를 한국어로 추출

In [5]:
def date_preprocess(df, date_colname, time_colname, new_colname):
    df_new = df.copy()
    
    # 먼저 시간, 분을 4자리 수로 변환 ex) 615->0615
    df_new[time_colname] = df_new[time_colname].astype(str).str.zfill(4)
    df_new[date_colname] = df_new[date_colname].astype(str)
    df_new[new_colname] = df_new[date_colname] + df_new[time_colname]
    # datetime화
    df_new[new_colname] = pd.to_datetime(df_new[new_colname], format='%Y%m%d%H%M')
    # 요일 만들기
    df_new[f'{new_colname}_day'] = df_new[new_colname].dt.day_name(locale='ko_kr') #locale 미설정시 영어로 나옴
    #기존 열들은 삭제
    df_new = df_new.drop([date_colname, time_colname], axis=1)
    return df_new

In [6]:
# TEST with 1000개 데이터
df1000 = df[:1000]
## 1. tran = tran_date + tran_time
df1000 = date_preprocess(df1000, 'tran_date', 'tran_time', 'tran')

## 2. play = play_date+ play_st_time
df1000 = date_preprocess(df1000, 'play_date', 'play_st_time', 'play')
df1000

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,seat,price,ticket_cancel,discount_type,performance_code,pre_open_date,open_date,genre,place,running_time,intermission,member_yn,tran,tran_day,play,play_day
0,50.0,F,골드,무료,그린,NaN,NaN,NaN,3층 BOX9 10,10000,2,일반,302,20220114.0,20220115.0,교향곡,콘서트홀,120,15,Y,2022-01-14 15:12:00,금요일,2022-02-04 20:00:00,금요일
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 7열 5,0,0,초대권,1120,NaN,20210823.0,독주,리사이틀홀,90,15,N,2021-10-03 15:50:00,일요일,2021-10-09 20:00:00,토요일
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 C블록 16열 3,0,0,초대권,2283,20220911.0,20220911.0,독주,IBK챔버홀,80,15,N,2022-09-17 15:20:00,토요일,2022-11-05 20:00:00,토요일
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 2열 3,0,0,초대권,2798,NaN,20190701.0,독주,리사이틀홀,90,15,N,2019-06-30 11:32:00,일요일,2019-07-31 20:00:00,수요일
4,50.0,M,무료,NaN,NaN,NaN,NaN,NaN,1층 B블록12열 7,180000,0,일반,897,20220109.0,20220110.0,독주,콘서트홀,90,0,Y,2022-02-06 16:15:00,일요일,2022-03-02 19:30:00,수요일
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 B블록22열 9,0,0,초대권,2732,NaN,20190419.0,클래식,콘서트홀,110,15,N,2019-07-11 15:04:00,목요일,2019-07-28 17:00:00,일요일
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 B블록 13열 15,0,0,기획사,2363,NaN,20230429.0,실내악,IBK챔버홀,100,15,N,2023-05-11 15:30:00,목요일,2023-06-10 20:00:00,토요일
997,40.0,F,무료,NaN,NaN,NaN,NaN,NaN,1층 E블록15열 11,60000,0,일반,311,NaN,20220626.0,클래식,콘서트홀,100,15,Y,2022-07-15 21:39:00,금요일,2022-12-31 17:00:00,토요일
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 B블록4열 3,0,0,초대권,1764,20190906.0,20190908.0,교향곡,콘서트홀,120,15,N,2019-10-03 13:25:00,목요일,2019-10-08 20:00:00,화요일


# 이제부터 df1000 데이터로 함수 확인
데이터가 너무 크기 때문에 아래 과정부터는 df1000을 사용하여 만든 함수를 확인하였음!

# 1-2. float_to_datetime
- float 타입의 데이터들을 datetime(yyyy-mm-dd 형식)으로 변경
- pre_open_date, open_date

In [7]:
def float_to_datetime(df, col_list):
    for col in col_list:
        df[col] = pd.to_datetime(df[col], format='%Y%m%d')

In [8]:
float_to_datetime(df1000, ['pre_open_date','open_date'])
df1000

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,seat,price,ticket_cancel,discount_type,performance_code,pre_open_date,open_date,genre,place,running_time,intermission,member_yn,tran,tran_day,play,play_day
0,50.0,F,골드,무료,그린,NaN,NaN,NaN,3층 BOX9 10,10000,2,일반,302,2022-01-14,2022-01-15,교향곡,콘서트홀,120,15,Y,2022-01-14 15:12:00,금요일,2022-02-04 20:00:00,금요일
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 7열 5,0,0,초대권,1120,NaT,2021-08-23,독주,리사이틀홀,90,15,N,2021-10-03 15:50:00,일요일,2021-10-09 20:00:00,토요일
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 C블록 16열 3,0,0,초대권,2283,2022-09-11,2022-09-11,독주,IBK챔버홀,80,15,N,2022-09-17 15:20:00,토요일,2022-11-05 20:00:00,토요일
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 2열 3,0,0,초대권,2798,NaT,2019-07-01,독주,리사이틀홀,90,15,N,2019-06-30 11:32:00,일요일,2019-07-31 20:00:00,수요일
4,50.0,M,무료,NaN,NaN,NaN,NaN,NaN,1층 B블록12열 7,180000,0,일반,897,2022-01-09,2022-01-10,독주,콘서트홀,90,0,Y,2022-02-06 16:15:00,일요일,2022-03-02 19:30:00,수요일
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 B블록22열 9,0,0,초대권,2732,NaT,2019-04-19,클래식,콘서트홀,110,15,N,2019-07-11 15:04:00,목요일,2019-07-28 17:00:00,일요일
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 B블록 13열 15,0,0,기획사,2363,NaT,2023-04-29,실내악,IBK챔버홀,100,15,N,2023-05-11 15:30:00,목요일,2023-06-10 20:00:00,토요일
997,40.0,F,무료,NaN,NaN,NaN,NaN,NaN,1층 E블록15열 11,60000,0,일반,311,NaT,2022-06-26,클래식,콘서트홀,100,15,Y,2022-07-15 21:39:00,금요일,2022-12-31 17:00:00,토요일
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 B블록4열 3,0,0,초대권,1764,2019-09-06,2019-09-08,교향곡,콘서트홀,120,15,N,2019-10-03 13:25:00,목요일,2019-10-08 20:00:00,화요일


# 1-3. tran_rank_min(df)
- 'performance_code'로 그룹화하여 'tran' 열을 가장 빠른 순서대로 정렬 후, rank를 매김
- 최종적으로 사용한 method = 'min' 이지만, method = 'first'와 비교하여 확인한 과정도 남겨두었음. 

In [9]:
# rank method = first
def tran_rank_first(df):    
    df['tran_rank'] = df.groupby('performance_code')['tran'].rank(ascending=True, method='first')

In [10]:
# df1000으로 TEST!
tran_rank_first(df1000)
df1000

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,seat,price,ticket_cancel,discount_type,performance_code,pre_open_date,open_date,genre,place,running_time,intermission,member_yn,tran,tran_day,play,play_day,tran_rank
0,50.0,F,골드,무료,그린,NaN,NaN,NaN,3층 BOX9 10,10000,2,일반,302,2022-01-14,2022-01-15,교향곡,콘서트홀,120,15,Y,2022-01-14 15:12:00,금요일,2022-02-04 20:00:00,금요일,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 7열 5,0,0,초대권,1120,NaT,2021-08-23,독주,리사이틀홀,90,15,N,2021-10-03 15:50:00,일요일,2021-10-09 20:00:00,토요일,1.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 C블록 16열 3,0,0,초대권,2283,2022-09-11,2022-09-11,독주,IBK챔버홀,80,15,N,2022-09-17 15:20:00,토요일,2022-11-05 20:00:00,토요일,1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 2열 3,0,0,초대권,2798,NaT,2019-07-01,독주,리사이틀홀,90,15,N,2019-06-30 11:32:00,일요일,2019-07-31 20:00:00,수요일,1.0
4,50.0,M,무료,NaN,NaN,NaN,NaN,NaN,1층 B블록12열 7,180000,0,일반,897,2022-01-09,2022-01-10,독주,콘서트홀,90,0,Y,2022-02-06 16:15:00,일요일,2022-03-02 19:30:00,수요일,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 B블록22열 9,0,0,초대권,2732,NaT,2019-04-19,클래식,콘서트홀,110,15,N,2019-07-11 15:04:00,목요일,2019-07-28 17:00:00,일요일,1.0
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 B블록 13열 15,0,0,기획사,2363,NaT,2023-04-29,실내악,IBK챔버홀,100,15,N,2023-05-11 15:30:00,목요일,2023-06-10 20:00:00,토요일,1.0
997,40.0,F,무료,NaN,NaN,NaN,NaN,NaN,1층 E블록15열 11,60000,0,일반,311,NaT,2022-06-26,클래식,콘서트홀,100,15,Y,2022-07-15 21:39:00,금요일,2022-12-31 17:00:00,토요일,1.0
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 B블록4열 3,0,0,초대권,1764,2019-09-06,2019-09-08,교향곡,콘서트홀,120,15,N,2019-10-03 13:25:00,목요일,2019-10-08 20:00:00,화요일,3.0


In [11]:
df1000.query("`performance_code`==1764")
# 순위가 매겨진 걸 확인해봤는데, method = 'first'를 사용하면 먼저 나타난 데이터에 우선순위를 부여함.
# 그러나 tran이 같은(즉, 동시에 예매한 경우) 경우가 존재. 
# 이 과정은 특정 공연에서 빨리 예매된 좌석이 어디인지를 파악하고 그 좌석이 좋은 좌석이라는 feature로 사용하고자 하므로
# method = 'min'을 사용해서 1,2,3이 아니라 1,1,3과 같이 순위를 매기기로 하였음.

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,seat,price,ticket_cancel,discount_type,performance_code,pre_open_date,open_date,genre,place,running_time,intermission,member_yn,tran,tran_day,play,play_day,tran_rank
305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2층 E블록6열 18,0,0,초대권,1764,2019-09-06,2019-09-08,교향곡,콘서트홀,120,15,N,2019-09-30 10:53:00,월요일,2019-10-08 20:00:00,화요일,1.0
572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3층 B블록7열 2,0,0,초대권,1764,2019-09-06,2019-09-08,교향곡,콘서트홀,120,15,N,2019-09-30 10:53:00,월요일,2019-10-08 20:00:00,화요일,2.0
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 B블록4열 3,0,0,초대권,1764,2019-09-06,2019-09-08,교향곡,콘서트홀,120,15,N,2019-10-03 13:25:00,목요일,2019-10-08 20:00:00,화요일,3.0


In [12]:
# rank method = min
def tran_rank_min(df):    
    df['tran_rank'] = df.groupby('performance_code')['tran'].rank(ascending=True, method='min')

In [13]:
# df1000으로 TEST하여 1,1,3과 같이 순위가 잘 매겨졌음을 확인함.
tran_rank_min(df1000)
df1000.query("`performance_code`==1764")

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,seat,price,ticket_cancel,discount_type,performance_code,pre_open_date,open_date,genre,place,running_time,intermission,member_yn,tran,tran_day,play,play_day,tran_rank
305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2층 E블록6열 18,0,0,초대권,1764,2019-09-06,2019-09-08,교향곡,콘서트홀,120,15,N,2019-09-30 10:53:00,월요일,2019-10-08 20:00:00,화요일,1.0
572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3층 B블록7열 2,0,0,초대권,1764,2019-09-06,2019-09-08,교향곡,콘서트홀,120,15,N,2019-09-30 10:53:00,월요일,2019-10-08 20:00:00,화요일,1.0
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1층 B블록4열 3,0,0,초대권,1764,2019-09-06,2019-09-08,교향곡,콘서트홀,120,15,N,2019-10-03 13:25:00,목요일,2019-10-08 20:00:00,화요일,3.0


# 2-1. separate_seat(df)
- 좌석 정보 분리하여 4개의 col에 저장 : seat_floor(층),seat_block(블록),seat_col(열),seat_num(좌석번호)  
- ChatGPT를 활용해 얻은 정규표현식 설명
  - df['seat'].str.extract(r'(\d+)층\s*([A-Z]*\w*)?(\d+열)?\s*(\d+)'):  
  - str.extract 메서드는 문자열을 추출하기 위한 Pandas Series의 문자열 메서드입니다.  
  - r'(\d+)층\s*([A-Z]*\w*)?(\d+열)?\s*(\d+)'는 정규식 패턴을 나타냅니다.  
  - (\d+)는 1개 이상의 숫자를 나타냅니다. 이 부분은 seat_floor (층)와 seat_num (좌석번호)에 해당합니다.  
  - 층 문자열은 입력 문자열에서 "층"이라는 글자를 정확히 일치시키는 역할을 합니다.  
  - ([A-Z]*\w*)?는 영대소문자와 숫자로 이루어진 문자열을 나타냅니다. 이 부분은 seat_block (블록)에 해당합니다.  
  - (\d+열)?는 1개 이상의 숫자 뒤에 "열"이라는 문자열이 나타날 수 있는 것을 나타냅니다. 이 부분은 seat_col (열)에 해당합니다.  
  - \s*는 공백 문자 (스페이스)가 0개 이상 나타날 수 있는 것을 나타냅니다. 이렇게 함으로써 숫자와 문자열 사이에 여분의 공백이 있어도 처리할 수 있습니다.  

In [14]:
df1000.seat.unique()
# unique 값을 확인해본 결과, 크게 아래 5가지 유형이 파악됨
# 3층 BOX9 10
# 2층 BOX4 1열 13
# 1층 7열 5
# 1층 C블록 16열 3
# 3층 G블록5열 8

array(['3층 BOX9 10', '1층 7열 5', '1층 C블록 16열 3', '1층 2열 3', '1층 B블록12열 7',
       '1층 A블록2열 1', '3층 E블록4열 8', '2층 D블록8열 4', '1층 B블록 11열 9',
       '1층 C블록17열 3', '2층 B블록 4열 3', '3층 BOX9 5', '1층 B블록 10열 6',
       '3층 BOX12 3', '2층 2열 5', '2층 D블록8열 3', '1층 C블록2열 2', '2층 E블록3열 16',
       '2층 BOX2 2', '1층 C블록20열 4', '1층 D블록10열 8', '1층 A블록 4열 2',
       '2층 BOX4 1열 13', '2층 3열 9', '1층 A블록 3열 2', '1층 4열 1', '1층 B블록7열 7',
       '3층 G블록5열 8', '1층 B블록21열 9', '1층 B블록17열 4', '1층 B블록 15열 2',
       '1층 D블록18열 6', '2층 A블록 6열 3', '3층 M블록1열 2', '1층 B블록 16열 9',
       '1층 8열 18', '1층 B블록 10열 4', '2층 B블록 1열 6', '1층 C블록15열 8',
       '1층 B블록3열 7', '2층 3열 19', '1층 B블록 11열 1', '3층 F블록5열 5',
       '2층 B블록 3열 3', '1층 E블록17열 1', '2층 B블록 2열 6', '1층 D블록7열 11',
       '1층 B블록 16열 7', '1층 7열 15', '1층 9열 13', '1층 D블록22열 3', '1층 4열 14',
       '3층 G블록1열 5', '1층 C블록4열 6', '1층 B블록7열 5', '1층 C블록 15열 5',
       '1층 8열 16', '1층 E블록19열 3', '2층 3열 6', '3층 F블록4열 3', '1층 C블록6열 7',
       '2층 C블록7열 7', '2층 A블록4열 7', '1층 

In [15]:
def separate_seat(df):    
    df[['seat_floor', 'seat_block', 'seat_col', 'seat_num']] = df['seat'].str.extract(r'(\d+)층\s*([A-Z]*\w*)?(\d+열)?\s*(\d+)')
    #seat_block에 공백기준 분리가 안 되어 있어서 'C블록4열', '3열'과 같이 붙어있던 데이터 제거
    df['seat_col'] = df['seat_block'].str.extract(r'(\d+열)', expand=False)
    df['seat_block'] = df['seat_block'].str.replace(r'\d+열', '', regex=True)
    #마지막으로 seat_block에서는 '블록', seat_col에서는 '열'이라는 글자 제거하기
    df['seat_block'] = df['seat_block'].str.replace('블록', '', regex=True)
    df['seat_col'] = df['seat_col'].str.replace('열', '', regex=True)
    df_new = df.drop(['seat'], axis=1)
    return df_new

In [16]:
df1000 = separate_seat(df1000)
df1000

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,price,ticket_cancel,discount_type,performance_code,pre_open_date,open_date,genre,place,running_time,intermission,member_yn,tran,tran_day,play,play_day,tran_rank,seat_floor,seat_block,seat_col,seat_num
0,50.0,F,골드,무료,그린,NaN,NaN,NaN,10000,2,일반,302,2022-01-14,2022-01-15,교향곡,콘서트홀,120,15,Y,2022-01-14 15:12:00,금요일,2022-02-04 20:00:00,금요일,1.0,3,BOX9,NaN,10
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,초대권,1120,NaT,2021-08-23,독주,리사이틀홀,90,15,N,2021-10-03 15:50:00,일요일,2021-10-09 20:00:00,토요일,1.0,1,,7,5
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,초대권,2283,2022-09-11,2022-09-11,독주,IBK챔버홀,80,15,N,2022-09-17 15:20:00,토요일,2022-11-05 20:00:00,토요일,1.0,1,C,NaN,16
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,초대권,2798,NaT,2019-07-01,독주,리사이틀홀,90,15,N,2019-06-30 11:32:00,일요일,2019-07-31 20:00:00,수요일,1.0,1,,2,3
4,50.0,M,무료,NaN,NaN,NaN,NaN,NaN,180000,0,일반,897,2022-01-09,2022-01-10,독주,콘서트홀,90,0,Y,2022-02-06 16:15:00,일요일,2022-03-02 19:30:00,수요일,1.0,1,B,12,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,초대권,2732,NaT,2019-04-19,클래식,콘서트홀,110,15,N,2019-07-11 15:04:00,목요일,2019-07-28 17:00:00,일요일,1.0,1,B,22,9
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,기획사,2363,NaT,2023-04-29,실내악,IBK챔버홀,100,15,N,2023-05-11 15:30:00,목요일,2023-06-10 20:00:00,토요일,1.0,1,B,NaN,13
997,40.0,F,무료,NaN,NaN,NaN,NaN,NaN,60000,0,일반,311,NaT,2022-06-26,클래식,콘서트홀,100,15,Y,2022-07-15 21:39:00,금요일,2022-12-31 17:00:00,토요일,1.0,1,E,15,11
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,초대권,1764,2019-09-06,2019-09-08,교향곡,콘서트홀,120,15,N,2019-10-03 13:25:00,목요일,2019-10-08 20:00:00,화요일,3.0,1,B,4,3


# 2-2. discount_type_sep()
- 일반 : 0%  
- 초대권 : 100% , 유의사항 : [초대권] 이런 게 있음.. '초대'라는 단어를 포함하고 있으면 항상 100% 할인일까? 확인 예정
- 그 외 : 각자 %
- nan : 기획사 등.. 지금 시점에서 정확한 %를 확인할 수 없는 경우

In [17]:
df['discount_type'].unique()

array(['일반', '초대권', '블루회원 할인20%', '골드회원 할인20%', '기획사',
       'K-lang 멤버십 회원(1인2매)20%', '기획사판매', '골드회원 할인30%', 'R석 기획사 할인',
       '싹틔우미 할인40%', '라흐마니노프탄생150주년(8매/3.27까지)20%', '공연진행석', '골드회원 할인10%',
       '장애인/국가유공자 할인50%', '싹딜가(골드)50%', '당일할인티켓', '그린회원 할인5%',
       '초,중,고,대학생(본인만)30%', '조기예매 할인40%', '그린회원 할인20%', '한화초대석',
       '단체100인이상50%', '공연예매권', '초/중/고등학생 할인(본인만)30%', '골드회원 할인15%',
       '골드회원 할인25%', '우리카드 우수회원', '조기예매 할인(1인8매/3월11일까지)30%', '신세계석',
       '골드회원 할인5%', '정기회원', '만65세 이상(본인만)50%', '싹딜가(블루)50%', '그린회원 할인15%',
       '직장인 할인(1인2매)20%', '블루회원 할인10%', '기획사판매20%',
       '초/중/고/대학생 할인(본인만)50%', '문화릴레이(1인2매/공연유료티켓제시)10%', '조기예매 할인10%',
       '노블회원 할인40%', 'K-Lang 할인(1인2매)20%', '골드회원 할인40%',
       '초/중/고등학생 할인(본인만)20%', '조기예매 할인(1인2매/~10월31일까지)30%', '기업석',
       '초/중/고/대학생 할인30%', '초/중/고/대학생 할인50%', '의사상자(동반1인/신분증지참)50%',
       '초,중,고,대학생(본인만)50%', '조기예매할인(1인8매/~6.30까지)30%', '그린회원 할인10%',
       '블루회원 할인5%', '연주자 할인50%', '스탭 할인30%', '홍보진행',
       'KOSYMI  유료회원(1

In [18]:
def discount_type_sep(df):
    # discount_name과 discount_rate 열 초기화
    df['discount_name'] = np.nan
    df['discount_rate(%)'] = np.nan

    # 조건에 따라 discount_name과 discount_rate 설정
    for idx, row in df.iterrows():
        if '일반' in row['discount_type']:
            df.at[idx, 'discount_name'] = '일반'
            df.at[idx, 'discount_rate(%)'] = '0'
        elif '초대권' in row['discount_type']:
            df.at[idx, 'discount_name'] = '초대권'
            df.at[idx, 'discount_rate(%)'] = '100'
        elif '%' in row['discount_type']:
            rate = ''.join(filter(str.isdigit, row['discount_type']))
            df.at[idx, 'discount_name'] = '기타'
            df.at[idx, 'discount_rate(%)'] = rate + ''

    # discount_type 열 제거
    df.drop('discount_type', axis=1, inplace=True)

In [19]:
discount_type_sep(df1000)
df1000

/var/folders/kg/xp1kxddd3r365qjsy8z_p9lh0000gn/T/ipykernel_3338/2441869586.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '일반' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'discount_name'] = '일반'
/var/folders/kg/xp1kxddd3r365qjsy8z_p9lh0000gn/T/ipykernel_3338/2441869586.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'discount_rate(%)'] = '0'


,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,price,ticket_cancel,performance_code,pre_open_date,open_date,genre,place,running_time,intermission,member_yn,tran,tran_day,play,play_day,tran_rank,seat_floor,seat_block,seat_col,seat_num,discount_name,discount_rate(%)
0,50.0,F,골드,무료,그린,NaN,NaN,NaN,10000,2,302,2022-01-14,2022-01-15,교향곡,콘서트홀,120,15,Y,2022-01-14 15:12:00,금요일,2022-02-04 20:00:00,금요일,1.0,3,BOX9,NaN,10,일반,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1120,NaT,2021-08-23,독주,리사이틀홀,90,15,N,2021-10-03 15:50:00,일요일,2021-10-09 20:00:00,토요일,1.0,1,,7,5,초대권,100
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2283,2022-09-11,2022-09-11,독주,IBK챔버홀,80,15,N,2022-09-17 15:20:00,토요일,2022-11-05 20:00:00,토요일,1.0,1,C,NaN,16,초대권,100
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2798,NaT,2019-07-01,독주,리사이틀홀,90,15,N,2019-06-30 11:32:00,일요일,2019-07-31 20:00:00,수요일,1.0,1,,2,3,초대권,100
4,50.0,M,무료,NaN,NaN,NaN,NaN,NaN,180000,0,897,2022-01-09,2022-01-10,독주,콘서트홀,90,0,Y,2022-02-06 16:15:00,일요일,2022-03-02 19:30:00,수요일,1.0,1,B,12,7,일반,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2732,NaT,2019-04-19,클래식,콘서트홀,110,15,N,2019-07-11 15:04:00,목요일,2019-07-28 17:00:00,일요일,1.0,1,B,22,9,초대권,100
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2363,NaT,2023-04-29,실내악,IBK챔버홀,100,15,N,2023-05-11 15:30:00,목요일,2023-06-10 20:00:00,토요일,1.0,1,B,NaN,13,NaN,NaN
997,40.0,F,무료,NaN,NaN,NaN,NaN,NaN,60000,0,311,NaT,2022-06-26,클래식,콘서트홀,100,15,Y,2022-07-15 21:39:00,금요일,2022-12-31 17:00:00,토요일,1.0,1,E,15,11,일반,0
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1764,2019-09-06,2019-09-08,교향곡,콘서트홀,120,15,N,2019-10-03 13:25:00,목요일,2019-10-08 20:00:00,화요일,3.0,1,B,4,3,초대권,100


# 작성예정

In [20]:
# column들의 데이터 타입을 int로 변경하는 함수
def dtype_to_int(df, col_list):
    for col in col_list:
        df[col] = df[col].astype('int')
        
# dtype_to_int(df, ['age', 'pre_open_date', 'open_date'])